In [1]:
import os
import pandas as pd

onlyWolf = pd.read_csv('./data/imagewoof2/noisy_imagewoof.csv')

In [3]:
dir ="./data/imagewoof2"
img_name = os.path.join(dir,onlyWolf.iloc[0, 0])
print(img_name)

./data/imagewoof2\train/n02115641/n02115641_3995.JPEG


In [2]:
import torchvision.models as models
from transformers import ViTModel,ViTConfig

model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

In [3]:
import torch

torch.save(model.state_dict(),"pretrained_weight.pth")

In [5]:
import torch
import torch.nn as nn
from torchvision.transforms import Resize, Compose
from transformers import ViTModel, ViTConfig

# ViT configuration
config = ViTConfig.from_pretrained('google/vit-base-patch16-224-in21k')  # 이미지넷으로 사전 학습된 모델의 configuration 불러오기

# 이미지 크기와 관련된 configuration 설정
config.image_size = (800, 1024)  # 이미지 크기 설정
config.num_patches = (800// 32) * (1024 // 32)  # 패치 개수 재설정
# 이미지넷으로 사전 학습된 모델의 가중치 불러오기
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k', config=config,ignore_mismatched_sizes=True)

# 이미지 전처리
transform = Compose([Resize(config.image_size)])




Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized because the shapes did not match:
- embeddings.position_embeddings: found shape torch.Size([1, 197, 768]) in the checkpoint and torch.Size([1, 3201, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, ToPILImage,Resize
from PIL import Image
import numpy as np
import torch
import pandas as pd
class lonelyWolfDataset(Dataset):
    """
    Load data from ./data/imagewoof2 
    read_csv file first.
    """
    def __init__(self, csv_file="./data/imagewoof2/noisy_imagewoof.csv", root_dir = "./data/imagewoof2", transform=None,is_valid = False,num_label = 10) -> None:
        super().__init__()
        df = pd.read_csv(csv_file)
        self.inform =  df[df['is_valid']== is_valid]
        self.root_dir = root_dir
        self.transform = transform
        self.is_valid = is_valid
        self.mapper = {}
    def __len__(self):
        return  len(self.inform)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = self.root_dir+"/"+self.inform.iloc[idx, 0]
        image = Image.open(img_name).convert("RGB")
        toTensor = ToTensor()
        reSize = Resize(size=(800,1024),)
        image = reSize(toTensor(image))
        #defalut dict 같은것 써도 무방
        label=0
        if self.inform.iloc[idx,1] in self.mapper:
            label = self.mapper[self.inform.iloc[idx,1]]
        else:
            label= len(self.mapper)
            self.mapper[self.inform.iloc[idx,1]] = label
        
        sample = {'image': image, 'label': label}

        return sample
    